# Dùng sentence bert cho bài toán searching dùng embedding (Question Answering).

Dùng code từ file:
https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb

Thuật toán search:
https://en.wikipedia.org/wiki/Okapi_BM25

In [1]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_25456/3777221515.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# This function will search all wikipedia articles for passages that
# answer the query

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

In [3]:
#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [4]:
import pandas as pd
from regress_adapter import get_data

data = get_data('TPB', '2022-01-01', '2023-09-29')
print(data.head())

/Users/ngocp/Documents/projects/pyml/botapp/algorithm/regress_adapter.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


                                                   text  close      date_y
3778  VNDIRECT: Hầu hết giá cổ phiếu của dệt may hiệ...  27680  2022-01-04
3777  Cách tốt nhất để các nhà đầu tư bảo vệ mình kh...  27680  2022-01-04
3776  Đảng uỷ cơ quan Uỷ ban Chứng khoán Nhà nước: S...  27680  2022-01-04
3775  Truyền hình K+ tặng đầu thu miễn phí đón ‘Tết ...  27680  2022-01-04
3774  Licogi 14 (L14) “bơm” thêm 214 tỷ đồng đầu tư ...  27190  2022-01-05


In [5]:
import helper
import numpy as np
from importlib import reload

reload(helper)

def clear_text_vi(texts = []):
    clear_ld = lambda t: helper.NLP(t).get_string_sanitize()
    t = map(clear_ld, texts)
    return list(t)

In [6]:
data_list = data['text'].values.tolist()
sent = clear_text_vi(data_list)

In [ ]:
# save sent to file use joblib
import joblib
joblib.dump(sent, './data/sba-sentences.pkl')


In [7]:
# sent = [
#     "This is an example sentence",
#     "Each sentence is converted bert embeddings",
#     "Sentences are passed as a list of string.",
#     "Cách tốt nhất để các nhà đầu tư bảo vệ mình",
# ]
corpus_embeddings = bi_encoder.encode(sent, convert_to_tensor=True, show_progress_bar=True)
print(len(corpus_embeddings))

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

5004


In [12]:
import joblib
joblib.dump(corpus_embeddings, './cached/sba-corpus_embeddings.pkl')

['./cached/sba-corpus_embeddings.pkl']

In [13]:

def search(query):
    print("Input question:", query)
    ##### Sematic Search #####
    query = clear_text_vi([query])[0]
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    # question_embedding = question_embedding
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, sent[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]


    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:3]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], sent[hit['corpus_id']].replace("\n", " ")))

In [14]:
search(query = "Tình hình chứng khoán thế giới ?")

Input question: Tình hình chứng khoán thế giới ?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	5.768	chứng_khoán tuần tình_hình đang xấu đithị trường_chứng_khoán thế_giới biến_động tiêu_cực các cổ_phiếu lớn đồng_loạt lao dốc và động_thái bán ròng của khối ngoại khiến tâm_lý nhà đầu_tư hoang_mang dẫn đến thị_trường giảm điểm mạnh vn-index rơi xuống dưới đường sma ngày cho thấy tình_hình càng bi_quan hơn i thị_trường chứng_khoán việt_nam tuần giao_dịch các chỉ_số chính giảm mạnh trong phiên cuối tuần kết phiên vn-index giảm điểm về mức điểm hnx-index giảm điểm kết phiên về mức điểm _xét cho cả tuần vn-index tổng_cộng giảm điểm hnx-index giảm điểm _thị_trường chứng_khoán ghi_nhận tuần giao_dịch đầy tiêu_cực với áp_lực bán vẫn còn khá mạnh các cây_nến đỏ chiếm số_lượng áp_đảo trong các phiên gần đây phiên cho thấy áp_lực bán mạnh đang hiện_diện riêng ngày giao_dịch cuối_cùng chỉ_số giảm điểm mạnh đã cắt xuống_đường sma ngày cho thấy tình_hình càng bi_quan hơn kết tuần vn-

In [10]:
search(query = "Giá kim loại, vàng miếng hôm nay ?")

Input question: Giá kim loại, vàng miếng hôm nay ?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	6.740	giá vàng hôm_nay vàng miếng sjc đắt hơn thế_giới triệu đồng lượng thanh_xuân xuanthanhnien@gmail.com gmt giá vàng miếng sjc ngày_càng đắt_đỏ so với thế_giới do kim_loại quý_quốc_tế liên_tục trượt giảm sâu còn vàng trong nước lại đứng ở mức cao arfasync push k911czvz sáng giá vàng miếng sjc tại eximbank giảm đồng mỗi lượng chiều mua vào còn triệu đồng và bán ra triệu đồng trong khi đó công_ty vàng_bạc đá_quý sài_gòn sjc giữ giá mua ở mức triệu đồng bán ra triệu đồng … ngược_lại giá vàng nhẫn số lại tăng nhẹ đồng mỗi lượng công_ty sjc mua vào với giá triệu đồng bán ra triệu đồng giá vàng trong nước neo ở mức cao trong những ngày qua và không thay_đổi nhiều so với đầu tháng trong khi giá thế_giới lại liên_tục đi xuống giảm usd ounce chính vì_vậy vàng miếng sjc ngày_càng có giá cao hơn quốc_tế lên triệu đồng lượng còn vàng nhẫn có giá cao hơn lên triệu đồng lượng giá vàn